# This notebook shows how to load asset or location meta data into Maximo
## First, upload data zip and scripts zip into ws project data assets.       
Make sure no space or special character in csv filename and the file path
+ HPU sample data in https://github.ibm.com/Watson-IoT/eam-hpu-lab/tree/main/csv-files    
+ Scripts in https://github.ibm.com/Watson-IoT/eam-hpu-lab/tree/main/scripts    

Commands example to compress the csv files and scripts.
E.g 
```
cd <csv file root path, /Users/gujuan/Documents/work/work/2022/lab/Github/eam-hpu-lab/csv-files>
zip -r hpu_csv_st.zip hpu_csv_st

cd <scripts root path, /Users/gujuan/Documents/work/work/2022/lab/Github/eam-hpu-lab/scripts>
zip -r hpu_dataloader.zip *.py
```

Note, if user wants to use hpu_csv_demo dataset, then please use below commands to compress the csv files. Change the second unzip command to unzip the hpu_csv_demo.zip, and also change DATAPATH to ```os.environ['DATA_PATH'] = '/tmp/hpu/csv/hpu_csv_demo'```

```
cd <csv file root path, /Users/gujuan/Documents/work/work/2022/lab/Github/eam-hpu-lab/csv-files>
zip -r hpu_csv_demo.zip hpu_csv_demo 

.............

!unzip /project_data/data_asset/hpu_csv_demo.zip -d /tmp/hpu/csv >> /dev/null

```



In [ ]:
# Clean previous 
!rm -rf /tmp/hpu/;mkdir -p /tmp/hpu/dataloader
# !cd /project_data/data_asset/;ls hpu_dataloader.zip

!unzip /project_data/data_asset/hpu_dataloader.zip -d /tmp/hpu/dataloader >> /dev/null
!unzip /project_data/data_asset/hpu_csv_st.zip -d /tmp/hpu/csv >> /dev/null
!ls /tmp/hpu/csv/hpu_csv_st
!ls /tmp/hpu/dataloader


import os
os.environ['DATA_PATH'] = '/tmp/hpu/csv/hpu_csv_st'
os.environ['DATA_LOADER_PATH'] = '/tmp/hpu/dataloader'

## Second, HPU dataloader uses HPU_Envs.json to store the environment varaibles, upload HPU_envs.json into ws project data assets. The format of the HPU_Envs.json is the following:
```
{
   "Instruction":{
      "MX_BASE_URL":"Extract health or manage host, and replace ******* in https://*******/maximo/, e.g Get main.manage.ivt13rel88.ivt.suite.maximo.com from https://main.manage.ivt13rel88.ivt.suite.maximo.com/maximo/oslc/graphite/relengineer/index.html",
      "MX_APIKEY":"Application Administration -> Go To Administration -> Integration -> API Keys -> Copy key from admin user card (e.g. 6805t46gn3tef37pu0picpg9vcq3hsmamm1enc43), or Add API key if not exist."
   },
   "MX_APIKEY":"**************",
   "MX_BASE_URL":"https://*******/maximo/"
}
```


Update os.environ['ORGID'],  os.environ['SITEID'] if user wants use a different organization and site.   
Update os.environ['THREADNUM'],  os.environ['BATCHSIZE'] if wants reduce the thread number or batchsize for each thread due to env performance issue.

In [ ]:
import json
# Opening JSON file
f = open('/project_data/data_asset/HPU_Envs.json',)
data = json.load(f)
f.close()
os.environ['MX_BASE_URL'] = data['MX_BASE_URL']
os.environ['MX_APIKEY'] = data['MX_APIKEY']

os.environ['ORGID'] = 'EUORG1'
os.environ['SITEID'] = 'EUDEMO'
os.environ['THREADNUM'] = '10'
os.environ['BATCHSIZE'] = '100'


## Third, run below cells to load data.

In [ ]:
!pip install requests_futures

### Create org and site if user wants to create a new organization and site.


In [ ]:
!python3 $DATA_LOADER_PATH/0_createOrgAndSite.py $ORGID $SITEID | tee log.txt
!if grep -q "'status': '400'" log.txt; then exit 1; fi

### Load classification
Update the classification sub-path to point classification csv files if you are using a different data structure compared to the sample data.


In [ ]:
!ls $DATA_PATH/1.classification

!python3 $DATA_LOADER_PATH/1_loadClassifications.py $ORGID $SITEID $DATA_PATH/1.classification | tee log.txt
!if grep -q "'status': '400'" log.txt; then exit 1; fi


### Load domain
#### Condition domain and ASSETTYPE domain
Update the domain sub-path to point domain csv files if you are using a different data structure compared to the sample data.



In [ ]:
!ls $DATA_PATH/2.domains

!python3 $DATA_LOADER_PATH/2_loadDomain.py $ORGID $SITEID $DATA_PATH/2.domains | tee log.txt
# !if grep -q "'status': '400'" log.txt; then exit 1; fi

### Load meters and meter groups
Update the meter group sub-path to point meter csv files if you are using a different data structure compared to the sample data.


In [ ]:
!ls $DATA_PATH/3.meters/meterGroups

!python3 $DATA_LOADER_PATH/3_loadMeters.py $ORGID $SITEID $DATA_PATH/3.meters/meterGroups | tee log.txt
!if grep -q "'status': '400'" log.txt; then exit 1; fi

### Load Locations
Update the location sub-path to point location csv files if you are using a different data structure compared to the sample data.


In [ ]:
!ls ${DATA_PATH}/4.locations/'4.0 container'

!python3 $DATA_LOADER_PATH/4_loadLocations.py $ORGID $SITEID ${DATA_PATH}/4.locations/'4.0 container' $THREADNUM $BATCHSIZE | tee log.txt
!if grep -q "'status': '400'" log.txt; then exit 1; fi

In [ ]:
!ls $DATA_PATH/4.locations

!python3 $DATA_LOADER_PATH/4_loadLocations.py $ORGID $SITEID $DATA_PATH/4.locations $THREADNUM $BATCHSIZE | tee log.txt
!if grep -q "'status': '400'" log.txt; then exit 1; fi

### Load Location  Meter Reading
Update the location meter reading sub-path to point location meter reading csv files if you are using a different data structure compared to the sample data.


In [ ]:
!ls ${DATA_PATH}/5n7.readings/5.locationMeterReading

!python3 $DATA_LOADER_PATH/6_loadReading.py $ORGID ${SITEID} ${DATA_PATH}/5n7.readings/5.locationMeterReading $THREADNUM $BATCHSIZE | tee log.txt
!if grep -q "'status': '400'" log.txt; then exit 1; fi

In [ ]:
# !ls ${DATA_PATH}/5n7.readings/5.locationMeterReading/tmp

### Load Assets
Update the asset sub-path to point asset csv files if you are using a different data structure compared to the sample data.


In [ ]:
!ls ${DATA_PATH}/6.assets

!python3 $DATA_LOADER_PATH/5_loadAsset.py $ORGID $SITEID ${DATA_PATH}/6.assets $THREADNUM $BATCHSIZE | tee log.txt
!if grep -q "'status': '400'" log.txt; then exit 1; fi

### Load Asset Meter Reading
Update the asset meter reading sub-path to point asset meter reading csv files if you are using a different data structure compared to the sample data.


In [ ]:
!ls ${DATA_PATH}/5n7.readings/7.assetMeterReading

!python3 $DATA_LOADER_PATH/6_loadReading.py $ORGID $SITEID ${DATA_PATH}/5n7.readings/7.assetMeterReading $THREADNUM $BATCHSIZE | tee log.txt
!if grep -q "'status': '400'" log.txt; then exit 1; fi

In [ ]:
# !ls /tmp/hpu/csv/csv_v87_st_only/5n7.readings/7.assetMeterReading/tmp

### Load LocSystem
Update the locsystem sub-path to point locsystem csv files if you are using a different data structure compared to the sample data.


In [ ]:
!ls ${DATA_PATH}/8.locSystem

!python3 $DATA_LOADER_PATH/7_loadLocSystem.py $ORGID $SITEID ${DATA_PATH}/8.locSystem $THREADNUM $BATCHSIZE | tee log.txt
!if grep -q "'status': '400'" log.txt; then exit 1; fi

### Update LocHierarchy
Update the location hierarchy sub-path to point location hierarchy csv files if you are using a different data structure compared to the sample data.


In [ ]:
!ls ${DATA_PATH}/9.containment

!python3 $DATA_LOADER_PATH/8_updateLocHierarchy.py $ORGID $SITEID ${DATA_PATH}/9.containment $THREADNUM $BATCHSIZE | tee log.txt
!if grep -q "'status': '400'" log.txt; then exit 1; fi

### Update Map Container Configuration
Update the map container configuraton sub-path to point map container configuraton  csv files if you are using a different data structure compared to the sample data.


In [ ]:
!ls ${DATA_PATH}/10.containerConfiguration

!python3 $DATA_LOADER_PATH/9_loadMapContainerCfg.py $ORGID $SITEID ${DATA_PATH}/10.containerConfiguration $THREADNUM $BATCHSIZE | tee log.txt
!if grep -q "'status': '400'" log.txt; then exit 1; fi